In [32]:
# Mandatory cell for the rest of this assignment

%load_ext autoreload
%autoreload 2

from sys import path

path.append('../scripts')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
def read_text_file(fileName):
    with open(fileName, 'r', encoding='utf-8') as f:
        return f.read()

In [34]:
from node import BlockchainNode
from network import Node
from wallet import Wallet
from proof_of_stake import ProofOfStake

walletNode1 = Wallet()
walletNode2 = Wallet()
walletNode3 = Wallet()

walletAlice = Wallet()
walletBob = Wallet()

WALLETS = {
    "Alice": walletAlice,
    "Bob": walletBob
}

pos = ProofOfStake(walletNode1.publicKey)

def reset_blockchain():
    Node.reset_network()
    global node1, node2, node3
    node1 = BlockchainNode(walletNode1, pos)
    node2 = BlockchainNode(walletNode2, pos)
    node3 = BlockchainNode(walletNode3, pos)

In [ ]:
import time
from certificate import Certificate
from helpers import timestamp
from smart_contracts.nft import SmartContract
from smart_contract import SmartContractDefinition, SmartContractWritingOperation

reset_blockchain()

smartContractDefinition = SmartContractDefinition(walletAlice.publicKey, read_text_file('smart_contracts/nft.py'), ["NFT", timestamp.now(),10, 20000 ])
walletAlice.sign(smartContractDefinition)

aliceMintOperation = SmartContract.mint_certificate(walletAlice.publicKey, smartContractDefinition)

walletAlice.sign(aliceMintOperation)

bobMintOperation = SmartContract.mint_certificate(walletBob.publicKey, smartContractDefinition)

walletBob.sign(bobMintOperation)

node1.new_certificate(smartContractDefinition)
node1.new_certificate(aliceMintOperation)
node1.new_certificate(bobMintOperation)

# wait 200 ms to try to mint after the mint time
# time.sleep(0.4)

aliceMintOperation = SmartContract.mint_certificate(walletAlice.publicKey, smartContractDefinition)

walletAlice.sign(aliceMintOperation)

node1.new_certificate(aliceMintOperation)

######################

for i in range(10):
    certificate = Certificate(walletAlice.publicKey)
    
    walletAlice.sign(certificate)
    node1.new_certificate(certificate)

######################


smartContractObject = SmartContractDefinition.get_smart_contract_at_current_state(
    node2.blockchain,
    smartContractDefinition.hash()
)


smartContractObject.display()

ValueError: non-hexadecimal number found in fromhex() arg at position 2860

In [ ]:
bobTransferOperation = SmartContract.transfer_certificate(walletBob.publicKey, smartContractDefinition, walletAlice.publicKey, 2)

walletAlice.sign(bobTransferOperation)

node2.new_certificate(bobTransferOperation)

bobTransferOperation = SmartContract.transfer_certificate(walletBob.publicKey, smartContractDefinition, walletBob.publicKey, 2)

walletBob.sign(bobTransferOperation)

node2.new_certificate(bobTransferOperation)

bobTransferOperation = SmartContract.transfer_certificate(walletBob.publicKey, smartContractDefinition, walletAlice.publicKey, 2)

walletBob.sign(bobTransferOperation)

node2.new_certificate(bobTransferOperation)


######################

for i in range(10):
    certificate = Certificate(walletAlice.publicKey)
    
    walletAlice.sign(certificate)
    node1.new_certificate(certificate)

######################

smartContractObject: SmartContract = SmartContractDefinition.get_smart_contract_at_current_state(
    node2.blockchain,
    smartContractDefinition.hash()
)


smartContractObject.display()


In [ ]:
from datetime import datetime, timedelta

def humanize_time(timestamp_ms):
    timestamp = datetime.fromtimestamp(timestamp_ms / 1000)  # Convert ms to seconds
    now = datetime.now()
    diff = now - timestamp

    if diff.total_seconds() < 0:  # Future time
        diff = abs(diff)
        suffix = "in "
    else:  # Past time
        suffix = ""
    
    if diff < timedelta(seconds=60):
        return f"{suffix}{int(diff.total_seconds())} seconds ago" if not suffix else f"{suffix}{int(diff.total_seconds())} seconds"
    elif diff < timedelta(minutes=60):
        return f"{suffix}{int(diff.total_seconds() / 60)} minutes ago" if not suffix else f"{suffix}{int(diff.total_seconds() / 60)} minutes"
    elif diff < timedelta(hours=24):
        return f"{suffix}{int(diff.total_seconds() / 3600)} hours ago" if not suffix else f"{suffix}{int(diff.total_seconds() / 3600)} hours"
    elif diff < timedelta(days=7):
        return f"{suffix}{int(diff.total_seconds() / 86400)} days ago" if not suffix else f"{suffix}{int(diff.total_seconds() / 86400)} days"
    else:
        return timestamp.strftime("%Y-%m-%d")


In [ ]:
%pip install flask
%pip install dominate

In [ ]:
from flask import Flask, jsonify, request
from dominate import document
from dominate.tags import *

from smart_contract import WritingOperationArguments



def fill_certificates(n=10):
    for i in range(n):
        certificate = Certificate(walletAlice.publicKey)
        
        walletAlice.sign(certificate)
        node1.new_certificate(certificate)

def get_smart_contract() -> SmartContract:
    fill_certificates()
    return SmartContractDefinition.get_smart_contract_at_current_state(
        node2.blockchain,
        smartContractDefinition.hash()
    )
    
def html_nft_view(token_id: int, publickey: str) -> str:
    nft_icon = smartContractObject.get_nft_icon(token_id)  # Get Unicode icon
    short_pubkey = publickey[256:264]  # Shortened public key

    # NFT card
    nft_card = div(cls="flex flex-col items-center justify-center w-32 h-40 bg-gray-100 rounded-xl shadow-md p-3")
    with nft_card:
        span(nft_icon, cls="text-4xl mb-2")  # NFT Icon
        p(f"Token {token_id}", cls="text-sm font-semibold text-gray-700")
        p(f"Owned by {short_pubkey}", cls="text-xs text-gray-500")
    return nft_card
    
def html_nfts_view(smartContract: SmartContract) -> str:
    div_nfts = div(id="nfts", cls="flex flex-wrap flex-row gap-4 p-4")  # Container for NFTs
    with div_nfts:
        h2("Minted NFTs", cls="w-full text-xl font-bold text-gray-800")  # Title

        for token_id, publickey in smartContract.token_owners.items():
            html_nft_view(token_id, publickey)
    
    return div_nfts

smartContractObject = get_smart_contract()
    
def calculate_end_mint_time(smartContract: SmartContract) -> int:
    return smartContract.start_mint_timestamp + smartContract.mint_time

def show_left_mint_time(smartContract: SmartContract) -> str:
    end_mint_time = calculate_end_mint_time(smartContract)
    return humanize_time(end_mint_time)


app = Flask(__name__)

@app.route('/')
def home():
    smartContractObject = get_smart_contract()
    doc = document(title="NFT Dashboard")
    
    with doc.head:
        title("NFT Dashboard")
        script(src="https://unpkg.com/htmx.org@2.0.4", integrity="sha384-HGfztofotfshcF7+8n44JQL2oJmowVChPTg48S+jvZoztPfvwD79OC/LTtG6dMp+", crossorigin="anonymous")
        script(src="https://unpkg.com/@tailwindcss/browser@4")

    with doc:
        with body(cls="bg-gray-100 p-6 min-h-screen flex flex-col items-center text-gray-800"):
            h1("NFT Dashboard", cls="text-3xl font-bold mb-6 text-gray-900")

            # Mint time left display
            mint_time_left_div = div(id="mint-time-left", cls="text-lg text-gray-700 mb-4 p-2 border border-gray-300 rounded")
            mint_time_left_div["hx-get"] = "/api/mint/timeleft"
            mint_time_left_div["hx-trigger"] = "load"
            mint_time_left_div["hx-swap"] = "outerHTML"

            # NFT Display Section
            div(cls="w-full max-w-4xl flex flex-wrap gap-4 justify-center")(
                html_nfts_view(smartContractObject)
            )

            # Wallet Selection Dropdown
            with div(cls="flex items-center gap-3 mt-6"):
                label("Select Wallet:", cls="font-medium text-gray-700")
                with select(id="wallet-select", name="wallet", cls="border rounded p-2 bg-white"):
                    for wallet_name in WALLETS.keys():
                        option(wallet_name, value=wallet_name)

            # Mint Button
            div(id="mint-error", cls="text-red-600 text-sm mt-2")
            with button("Mint NFT", id="mint-btn", 
                        cls="mt-4 bg-blue-600 hover:bg-blue-700 text-white font-semibold py-2 px-4 rounded-lg shadow") as b:
                b["hx-post"] = "/mint"
                b["hx-target"] = "#nfts"
                b["hx-swap"] = "beforeend" 
                b["hx-include"] = "#wallet-select"
                script("""
                    document.body.addEventListener('htmx:responseError', function(event) {
                        if (event.detail.elt.id !== 'mint-btn') return;
                        document.getElementById('mint-error').innerHTML = event.detail.xhr.responseText;
                    });

                    document.body.addEventListener('htmx:afterRequest', function(event) {
                        if (event.detail.elt.id !== 'mint-btn') return;
                        if (event.detail.successful) {
                            document.getElementById('mint-error').innerHTML = '';  // Clear errors on success
                        }
                    });
                """)

            # Transfer Form
            div(id="transfer-error", cls="text-red-600 text-sm mt-2")
            with form(id="transfer-form", cls="mt-6 flex flex-col gap-3 bg-white shadow-md p-4 rounded-lg w-full max-w-md") as f:
                label("Transfer NFT:", cls="text-lg font-semibold text-gray-700")
                f["hx-post"] = "/transfer"
                f["hx-target"] = "#nfts"
                f["hx-swap"] = "outerHTML"
                f["hx-include"] = "#wallet-select"

                with select(name="to_wallet", required=True, cls="border rounded p-2 bg-white"):
                    for wallet_name in WALLETS.keys():
                        option(wallet_name, value=wallet_name)

                input_(type="number", name="token", placeholder="Token ID", required=True, min=0, max=smartContractObject.token_id, cls="border rounded p-2")

                button("Transfer", type="submit", 
                       cls="bg-green-600 hover:bg-green-700 text-white font-semibold py-2 px-4 rounded-lg shadow")
                script("""
                    document.body.addEventListener('htmx:responseError', function(event) {
                        if (event.detail.elt.id !== 'transfer-form') return;
                        document.getElementById('transfer-error').innerHTML = event.detail.xhr.responseText;
                    });
                    document.body.addEventListener('htmx:afterRequest', function(event) {
                        if (event.detail.elt.id !== 'transfer-form') return;
                        if (event.detail.successful) {
                            document.getElementById('transfer-error').innerHTML = '';  // Clear errors on success
                        }
                    });
                """)

    return doc.render()

@app.route('/mint', methods=['POST'])
def mint():
    smartContractObject = get_smart_contract()
    # Get selected wallet from form data
    selected_wallet_name = request.form.get("wallet")
    selected_wallet = WALLETS.get(selected_wallet_name)

    if not selected_wallet:
        return div("Error: Invalid wallet selected", cls="error-message").render(), 400
    
    # Check the operation locally
    new_token, err = smartContractObject.mint(WritingOperationArguments(selected_wallet.publicKey, timestamp.now()))
    if err:
        error_div = div(f"Error: {err}", cls="error-message")
        return error_div.render(), 400 
    # Add the operation to the blockchain
    mint_operation = SmartContract.mint_certificate(selected_wallet.publicKey, smartContractDefinition)
    selected_wallet.sign(mint_operation)
    node1.new_certificate(mint_operation)
    return html_nft_view(new_token, selected_wallet.publicKey).render()

@app.route('/transfer', methods=['POST'])
def transfer():
    smartContractObject = get_smart_contract()
    # Get selected wallet from form data
    selected_wallet_name = request.form.get("wallet")
    selected_wallet = WALLETS.get(selected_wallet_name)

    if not selected_wallet:
        return div("Error: Invalid wallet selected", cls="error-message").render(), 400

    # Get transfer data from form
    to_wallet_name = request.form.get("to_wallet")
    to_wallet = WALLETS.get(to_wallet_name)
    token = int(request.form.get("token"))

    if not to_wallet:
        return div("Error: Invalid wallet to transfer to", cls="error-message").render(), 400

    # Check the operation locally
    res, err = smartContractObject.transfer( to_wallet.publicKey,token, WritingOperationArguments(selected_wallet.publicKey, timestamp.now()))
    if err:
        error_div = div(f"Error: {err}", cls="error-message")
        return error_div.render(), 400

    # Add the operation to the blockchain
    transfer_operation = SmartContract.transfer_certificate(selected_wallet.publicKey, smartContractDefinition, to_wallet.publicKey, token)
    selected_wallet.sign(transfer_operation)
    node1.new_certificate(transfer_operation)
    return html_nfts_view(smartContractObject).render()

@app.route('/api/mint/timeleft')
def mint_time_left():
    mint_time_left = calculate_end_mint_time(smartContractObject) - timestamp.now()
    
    mint_time_left_div = div(id="mint-time-left", cls="p-3 rounded-lg border bg-white shadow-md text-gray-800 text-lg text-center w-full max-w-sm")
    mint_time_left_div["hx-get"] = "/api/mint/timeleft"
    mint_time_left_div["hx-swap"] = "outerHTML"
    
    if abs(mint_time_left) < 60000:
        mint_time_left_div["hx-trigger"] = "every 1s"
    elif abs(mint_time_left) < 300000:
        mint_time_left_div["hx-trigger"] = "every 10s"
    else:
        mint_time_left_div["hx-trigger"] = "every 60s"

    mint_time_left_div.add(
        span(f"Left mint time: {show_left_mint_time(smartContractObject)}", cls="font-semibold text-blue-600")
    )
    
    return mint_time_left_div.render()
    
    # return f"{p(f"Left mint time: {show_left_mint_time(smartContractObject)}")}"

if __name__ == '__main__':
    app.run(port=8000)


In [ ]:
from flask import Flask
from dominate import document
from dominate.tags import *

app = Flask(__name__)

@app.route('/')
def home():
    smartContractObject: SmartContract = SmartContractDefinition.get_smart_contract_at_current_state(
        node2.blockchain,
        smartContractDefinition.hash()
    )
    doc = document(title="My Page")
    with doc.head:
        title("My Page")
        script(src="https://unpkg.com/htmx.org@2.0.4", integrity="sha384-HGfztofotfshcF7+8n44JQL2oJmowVChPTg48S+jvZoztPfvwD79OC/LTtG6dMp+", crossorigin="anonymous")
    with doc:
        with body():
            with div(id="nfts"):
                # show all minted nfts
                h1("Minted NFTs")
                for token, publickey in smartContractObject.token_owners.items():
                    p(f"Token {token} owned by {publickey}")
                    
                
            
    return doc.render()

@app.route('/state')
def state():
    return smartContractObject.html()

@app.route('/a')
def a():
    return "You clicked the button!"

if __name__ == '__main__':
    app.run(port=8000)